![alt text](FGV_logo.png)

# Machine Learning

## Regressão Logistica

In [ ]:
# import de modulos pandas e numpy
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

# Math
import math

# import de modulos para graficos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# imports para Machine Learning 
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split # modulo antigo
from sklearn.model_selection import train_test_split

# para avaliacao
from sklearn import metrics

# dataset
import statsmodels.api as sm

### Função logística

A função logistica [Logistic Function](http://en.wikipedia.org/wiki/Logistic_function) recebe como argumento uma valor de $-\infty$ a $+\infty$, e retorna um valor no intervalo $(0,1)$

$$ \sigma (t)= \frac{1}{1+e^{-t}}$$

In [ ]:
# Função Logistica
def logistic(t):
    return 1.0 / (1 + math.exp((-1.0)*t) )

# cria um grade de -6 a 6 ( 500 elementos, igualmente espaçados)
x_grade = np.linspace(-6,6,500)

# calcula os valores de y
y = np.array([logistic(x) for x in x_grade])

# usando exp do numpy que aceita um vetor como argumento
y = 1/(1 + np.exp(-1.0 * x_grade))

# plot
plt.plot(x_grade,y)
plt.title(' Funcao Logistica ')

A ideia é considerar a função logistica gerando uma probabilidade, a partir de um polinomio:


$$ y = a_0 + a_1.x_{1} + a_2.x_{2} ... + a_m.x_{m}$$

onde $a_0, a_1, ..., a_m$ são coeficientes a serem aprendidos, de forma que a equação abaixo:<br>


$$ F(x)= \frac{1}{1+e^{-(a_0 + a_1.x_{1} + a_2.x_{2} ... + a_m.x_{m})}}$$

forneça a 'melhor' probabilidade de sucesso. 

Para tanto o algoritmo procura encontrar os melhores $a_0, a_1, ..., a_m$ que minimizam o erro. 

Na predição, o $x^{(i)}$ fornece um $F(x^{(i)})$. 
* Se $F(x^{(i)}) \leq 0.5$, predição será a classe 0
* Se $F(x^{(i)}) > 0.5$, predição será a classe 1

### Exemplo prático

In [ ]:
import webbrowser

url = 'http://statsmodels.sourceforge.net/stable/datasets/generated/fair.html'

webbrowser.open_new(url)

Quantidade de observações: 6366
Quantidade de features: 9
Definições:

    rate_marriage   : How rate marriage, 1 = very poor, 2 = poor, 3 = fair,
                    4 = good, 5 = very good
    age             : Age
    yrs_married     : No. years married. Interval approximations. See
                    original paper for detailed explanation.
    children        : No. children
    religious       : How relgious, 1 = not, 2 = mildly, 3 = fairly,
                    4 = strongly
    educ            : Level of education, 9 = grade school, 12 = high
                    school, 14 = some college, 16 = college graduate,
                    17 = some graduate school, 20 = advanced degree
    occupation      : 1 = student, 2 = farming, agriculture; semi-skilled,
                    or unskilled worker; 3 = white-colloar; 4 = teacher
                    counselor social worker, nurse; artist, writers;
                    technician, skilled worker, 5 = managerial,
                    administrative, business, 6 = professional with
                    advanced degree
    occupation_husb : Husband's occupation. Same as occupation.
    affairs         : measure of time spent in extramarital affairs


In [ ]:
# Carregando dataframe 

df = sm.datasets.fair.load_pandas().data

In [ ]:
df.sample(5)

In [ ]:
# Define uma coluna, indicador binário que indica se a pessoa teve um caso extra-conjugal
df['teve_affair'] = [ 1 if a else 0 for a in df.affairs]

In [ ]:
df.sample(5)

In [ ]:
# Media por cada label
df.groupby('teve_affair').mean()

In [ ]:
# desvio padrão por cada label
df.groupby('teve_affair').std()

In [ ]:
sns.set_context('poster')

In [ ]:
# sns.jointplot(data = df[df.affairs < 2], x = 'age', y = 'affairs')

In [ ]:
df.affairs.describe(percentiles=[0, 0.9, 0.99])

In [ ]:
# Countplot com idade
sns.countplot(data=df, x='age', hue='teve_affair', palette='coolwarm')
# sns.distplot(df[df.teve_affair == 0].age)
# sns.distplot(df[df.teve_affair == 1].age)

In [ ]:
# Countplot com anos de casamento
sns.countplot('yrs_married',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# Countplot com quantidade de filhos
sns.countplot('children',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# Countplot com nivel de educação
sns.countplot('educ',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# considerando que a pessoa teve um caso extra-conjugal, 
# visualizacão da distribuição por boxplot
sns.boxplot(data=df[df.affairs > 0], x='age', y = 'affairs')
plt.ylim(0, 15)

In [ ]:
# considerando que a pessoa teve um caso extra-conjugal, 
# visualizacão da distribuição por violinplot
sns.violinplot(data=df[(df.affairs > 0) & (df.affairs < 9)], x='age', y = 'affairs')
plt.ylim(-2, 12)

In [ ]:
sns.pairplot(data=df[['affairs', 'age', 'yrs_married', 'children', 'religious']], 
        plot_kws = {'alpha':0.08, 's':180, 'edgecolor': None}, )

#### pre-processamento

Notamos as variaveis categoricas Occupation e Husband's Occupation. De maneira similar a regressão linear, precisamos tratar essas colunas. Nesse caso, fazemos um OHE (one hot encoding). Importante: Labelencoding não funciona!!

Pandas tem um método para criar esses [dummy variables](http://en.wikipedia.org/wiki/Dummy_variable_%28statistics%29) criando colunas dedicadas para cada valor encontrado.

In [ ]:
# Cria novo DataFrame para as colunas categoricas
occ_dummies = pd.get_dummies(df['occupation'])
hus_occ_dummies = pd.get_dummies(df['occupation_husb'])

occ_dummies.head()

In [ ]:
# Atribui nomes as colunas
occ_dummies.columns = ['occ1','occ2','occ3','occ4','occ5','occ6']
hus_occ_dummies.columns = ['hocc1','hocc2','hocc3','hocc4','hocc5','hocc6']

In [ ]:
# Atribui X primeiramente sem as colunas categoricas 
X = df.drop(['occupation','occupation_husb','teve_affair'],axis=1)

# Concatena os dataframes dummies
dummies = pd.concat([occ_dummies,hus_occ_dummies],axis=1)

In [ ]:
# Concatena o X com o dataframe dos dummies
X = pd.concat([X,dummies],axis=1)

# amostragem do X
X.sample(5)

#### multicolinearidade

Notar que nos dataframe com as variaveis dummies, uma das colunas é combinação linear das outras.

Por exemplo, o valor da primeira coluna será igual a $1 - \sum_{i=2}^{k} x_i$, onde $x_i$ é o valor da coluna $i$ na mesma observação, e $k$ é a cardinalidade (quantidade de valores possíveis) da coluna categorica original.

Para remediar esse problema, bem simples, basta deletar uma das colunas.

In [ ]:
# deletando uma coluna para cada coluna categorica
X = X.drop('occ1',axis=1)
X = X.drop('hocc1',axis=1)

# deletando coluna não utilizada
X = X.drop('affairs',axis=1)

# amostragem
X.sample(5)

#### alternativa
Por razões didáticas o roteiro acima foi apresentado, mas todo ele pode ser resumido em apenas uma linha:

In [ ]:
# X = pd.get_dummies(df, columns=['occupation', 'occupation_husb'], 
#                    drop_first=True).drop(['affairs', 'teve_affair'], axis = 1)

#### separando o vetor y 
Agora que já temos a matriz com as features definidas, vamos retirar a coluna resposta em um vetor a parte ($y$)

In [ ]:
# Atribui y a coluna teve_affair
y = df.teve_affair

# amostragem do y
y.head()

In [ ]:
# transformando em numpy
y = y.values

# checando resultado
y

In [ ]:
y[3683]

#### Rodando Regressão Logistica com sklearn 

In [ ]:
# Instanciando objeto
log_model = LogisticRegression()

# Treinando o modelo
log_model.fit(X, y)

# Checando acurácia
log_model.score(X, y)

In [ ]:
# Checando a percentagem de pessoas com casos extra-conjugais
y.mean()

In [ ]:
# coeff_df = DataFrame(X.columns, np.transpose(list(log_model.coef_))))

In [ ]:
coeff_df = DataFrame(log_model.coef_)

coeff_df.columns = X.columns

coeff_df = coeff_df.T

coeff_df.columns = ['coeficiente']

In [ ]:
coeff_df.plot(kind='bar', figsize=(12,6))

##### O que podemos observar pelos coeficientes acima?

### Treinamento e validação

In [ ]:
# Segmentando a base
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Instanciando um objeto
log_model2 = LogisticRegression()

# Treinando o modelo
log_model2.fit(X_train, y_train)

In [ ]:
# Predizendo a classe das observações de teste
class_predict = log_model2.predict(X_test)

# Comparando as classes da predição e o gold, 
# ou seja, medindo a performance...
print (metrics.accuracy_score(y_test,class_predict))

### Analise de resultados via matriz de confusão

In [ ]:
# matriz de confusão
metrics.confusion_matrix(y_test, class_predict)

In [ ]:
# Label gold na vertical, e label predição na horizontal
sns.heatmap(metrics.confusion_matrix(y_test, class_predict), annot=True, fmt ='d')

In [ ]:
print(metrics.classification_report(y_test, class_predict))

#### Quanto devemos acertar....

In [5]:
# se não sabemos nada da pessoa, o melhor 'chute' seria jogar uma moeda com 30% de probabilidade de cara (label = 1)
# 30% é a media do vetor y (arredondado)

# nesse caso teriamos:
print('prob. verdadeiro positivo:', 0.3 * 0.3)
print('prob. verdadeiro negativo:', 0.7 * 0.7)
print('prob. falso positivo:', 0.7 * 0.3)
print('prob. falso negativo:', 0.3 * 0.7)

prob. verdadeiro positivo: 0.09
prob. verdadeiro negativo: 0.48999999999999994
prob. falso positivo: 0.21
prob. falso negativo: 0.21


In [7]:
print('acuracia', 0.3 * 0.3 + 0.7 * 0.7)

acuracia 0.58


In [9]:
# Na realidade esse não é o melhor chute para maximizar a acuracia. 
# para maximizar a acuracia, o melhor chute seria 'chutar' tudo 0. 
# nesse caso teriamos:

print('prob. verdadeiro positivo:', 0.3 * 0)
print('prob. verdadeiro negativo:', 0.7 * 1)
print('prob. falso positivo:', 0.7 * 0)
print('prob. falso negativo:', 0.3 * 1)

prob. verdadeiro positivo: 0.0
prob. verdadeiro negativo: 0.7
prob. falso positivo: 0.0
prob. falso negativo: 0.3


In [10]:
print('acuracia', 0.3 * 0 + 0.7 * 1)

acuracia 0.7
